In [17]:
!git clone https://github.com/tsebaka/ML-from-scratch
import sys
sys.path.insert(0,'/content/ML-from-scratch/Parse-lib')

fatal: destination path 'ML-from-scratch' already exists and is not an empty directory.


In [18]:
import numpy as np
import nltk
import bs4 as bs
import re
import urllib.request
import warnings
import pp

from nltk.corpus import stopwords

warnings.filterwarnings('ignore')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Этот ноутбук будет дополнением к word2vec который был в курсе [NLP for you](https://lena-voita.github.io/nlp_course/word_embeddings.html) в главе Word Embeddings. Дополню я часть в которой берётся градиент от [Negative Log-Likelihood](https://github.com/testpassword/Machine-learning-and-data-analysis/blob/master/5%20-%20Логистическая%20регрессия/5.Логистическая_Регрессия.pdf), а точнее не берётся, это [Лена](https://github.com/lena-voita) оставила в качестве домашнего задания. Градиент по сути не сложный, но запутаться легко, да и к тому же при его взятии возникает один вопрос о котором мы и поговорим. Также считаю нужным объяснить смысл самой лосс функции: "почему скалярное произведение?".

Начнём со смысла лосс функции. Вспомним для начала, что мы вообще хотим от наших эмбеддингов. Мы хотим чтобы слова похожие по смыслу, были близки друг к другу, то есть имели похожие векторные представления. Вспомним, что при фиксированной длине при увеличении скалярного произведение, расстояние между векторами уменьшается: это всё равно что уменьшать угол между векторами, так как при стремлении угла к 0, косинус угла увеличивается -> увеличивается скалярное произведение. Осталось подумать как не дать нашим векторам увеличивать скалярное произведение не за счёт увеличения своей длинны (в таком случае они будут отдаляться друг от друга), а за счёт уменьшения угла. Тут можно использовать [L2 регуляризацию](https://ml-handbook.ru/chapters/linear_models/intro#регуляризация).

$$L(U, V) = -u_{context} \cdot v_{central} + \log{\sum{_{w\in V}} \exp{(u_w \cdot v_{central})}} $$

$$\frac{\partial L}{\partial v_{central}} = -u_{central} + \frac{1}{\sum_{w \in V}{\exp{u_w \cdot v_{central}}}} \cdot \sum_{w \in V}[\exp{(u_w \cdot v_{central})}\cdot u_{w}] $$

Если $ w = context \text{,}$ тогда градиент равен

$$ \frac{\partial L}{\partial u_{w}}=  -u_{central} +\frac{1}{\sum_{w \in V}{\exp({u_w v_{central}})}} \cdot \exp{(u_{context}  v_{central})}\cdot v_{central}  $$


при $w \neq context \text{:}$

$$ \frac{\partial L}{\partial u_{w}}=  \frac{1}{\sum_{w \in V}{\exp({u_w v_{central}})}} \cdot \exp{(u_w  v_{central})}\cdot v_{central}  $$


# **Word2vec**

In [19]:
sites = [#'https://breakingbad.fandom.com/wiki/Walter_White',
         'https://en.wikipedia.org/wiki/Breaking_Bad',
        #  'https://breakingbad.fandom.com/wiki/Jesse_Pinkman',
        #  'https://breakingbad.fandom.com/wiki/Gustavo_Fring', 
        # 'https://breakingbad.fandom.com/wiki/Jimmy_McGill',
        # 'https://breakingbad.fandom.com/wiki/Mike_Ehrmantraut',
        # 'https://breakingbad.fandom.com/wiki/Skyler_White',
        # 'https://breakingbad.fandom.com/wiki/Hank_Schrader'
         ]

parser = pp.parse_and_prepare()
vocab, text = parser.parse(sites)

In [74]:
class word2vec():
    def __init__(self, iterations=5, learning_rate=0.1, l2=0.01, window=3, negative=5, vector_size=300):
        self.iterations = iterations
        self.learning_rate = learning_rate
        self.l2 = l2
        self.negative = negative
        self.vector_size = vector_size
        self.window = window

        self.V, self.U = self.init_weights()

    def init_weights(self):
        V = np.random.uniform(-0.6, 0.6, (len(vocab), self.vector_size))
        U = np.random.uniform(-0.6, 0.6, (self.vector_size, len(vocab)))
        return V, U

    def computeGradient(self, central_vec, negative_sample): #
        flag = 0
        sum_exp_dot_cont = 0
        for sample in negative_sample:
            exponents = np.array([])
            sample = np.array(sample)
            for i in range(self.negative):
                temp = np.exp(np.dot(sample[:, i], central_vec))
                exponents = np.append(exponents, temp)
                if flag == 0:
                    sum_exp_dot_cont += temp * sample[:, i]

            sum_exp = np.sum(exponents)
            if flag == 0:
                grad_central = (1 / sum_exp) * sum_exp_dot_cont - sample[:, self.negative - 1]
                flag = 1

            grad_negative_sample = []
            grad_negative = []

            for i in range(self.negative):
                grad_negative.append((1 / sum_exp) * np.exp(np.dot(sample[:, i], central_vec)) * central_vec)

            grad_negative[self.negative - 1] += -central_vec
            grad_negative_sample.append(grad_negative)

        return grad_central, grad_negative_sample

    def computeRegularization(self, z):
        # l2 reg for prohibit vectors from increasing the scalar product by 
        # increasing the size of the vector (that is, they can be separated)
        return 2 * self.l2 * z
        
    def get_pos_in_vocab(self, vocab, word):
        try:
            for pos in range(len(vocab)):
                if vocab[pos] == word:
                    return pos
        except ValueError:
            print("This word isn't in dictionary:", word)
    
    def get_negative_sample(self, context):
        negative_sample = []
        for i in range(len(context)):
            random_index_sample = np.random.randint(self.vocab_size, size=self.negative)
            random_index_sample[self.negative - 1] = context[i][1]
            negative_sample_for_one = []
            for k in range(self.negative):
                negative_sample_for_one.append(self.U[:, random_index_sample[k]])  
            negative_sample.append([negative_sample_for_one])
            
        return negative_sample

    def get_context(self, text, pos_central):
        context = []
        cnt = 1
        while cnt <= self.window and pos_central + cnt <= len(text):
            context.append([text[pos_central+cnt], pos_central+cnt])
            cnt = cnt + 1
        cnt = 1
        while cnt <= self.window and pos_central - cnt >= 0:
            context.append([text[pos_central-cnt], pos_central-cnt])
            cnt = cnt + 1
        
        return context

    def update_weights(self, grad_central, grad_negative_sample, pos_central, context):
        self.V[pos_central] -= self.learning_rate * grad_central + self.computeRegularization(self.V[pos_central])
        for i in range(context):
            self.U[:, context[i][1]] -= self.learning_rate * grad_negative_sample[i] + self.computeRegularization(self.U[:, pos_central])

    def fit(self, vocab, text):
        self.vocab_size = len(vocab)
        for iteration in range(self.iterations):
            for central_index, word in enumerate(text):
                pos_central = self.get_pos_in_vocab(vocab, word)
                pos_central = 4
                context = self.get_context(text, pos_central) # 
                negative_sample = self.get_negative_sample(context)

                grad_central, grad_negative_sample = self.computeGradient(self.V[pos_central], negative_sample)
                self.update_weights(grad_central, grad_negative_sample, pos_central, context)
        return self.U, self.V

PPMI compare

In [ ]:
wv = word2vec()
con = wv.fit(vocab, text)